In [ ]:
## import packages
from datetime import datetime
import os
import pandas as pd
import numpy as np
import random
from itertools import chain
import hail as hl
from hail.linalg import BlockMatrix

In [ ]:
DATASET = os.getenv('WORKSPACE_CDR')
bucket = os.getenv('WORKSPACE_BUCKET')

## Download the summary statistics for 10 phenotypes

In [10]:
!curl https://pan-ukb-us-east-1.s3.amazonaws.com/sumstats_flat_files/continuous-50-both_sexes-irnt.tsv.bgz | gsutil cp - "gs://fc-secure-9afe7562-2fad-4781-ab60-03528a626c19/Sumstats/continuous-50-both_sexes-irnt.tsv.bgz"
!curl https://pan-ukb-us-east-1.s3.amazonaws.com/sumstats_flat_files/continuous-DBP-both_sexes-auto_medadj_irnt.tsv.bgz | gsutil cp - "gs://fc-secure-9afe7562-2fad-4781-ab60-03528a626c19/Sumstats/continuous-DBP-both_sexes-auto_medadj_irnt.tsv.bgz"
!curl https://pan-ukb-us-east-1.s3.amazonaws.com/sumstats_flat_files/biomarkers-30760-both_sexes-irnt.tsv.bgz | gsutil cp - "gs://fc-secure-9afe7562-2fad-4781-ab60-03528a626c19/Sumstats/biomarkers-30760-both_sexes-irnt.tsv.bgz"
!curl https://pan-ukb-us-east-1.s3.amazonaws.com/sumstats_flat_files/biomarkers-30690-both_sexes-irnt.tsv.bgz | gsutil cp - "gs://fc-secure-9afe7562-2fad-4781-ab60-03528a626c19/Sumstats/biomarkers-30690-both_sexes-irnt.tsv.bgz"
!curl https://pan-ukb-us-east-1.s3.amazonaws.com/sumstats_flat_files/continuous-30010-both_sexes-irnt.tsv.bgz | gsutil cp - "gs://fc-secure-9afe7562-2fad-4781-ab60-03528a626c19/Sumstats/continuous-30010-both_sexes-irnt.tsv.bgz"
!curl https://pan-ukb-us-east-1.s3.amazonaws.com/sumstats_flat_files/continuous-30000-both_sexes-irnt.tsv.bgz | gsutil cp - "gs://fc-secure-9afe7562-2fad-4781-ab60-03528a626c19/Sumstats/continuous-30000-both_sexes-irnt.tsv.bgz"
!curl https://pan-ukb-us-east-1.s3.amazonaws.com/sumstats_flat_files/phecode-250.2-both_sexes.tsv.bgz | gsutil cp - "gs://fc-secure-9afe7562-2fad-4781-ab60-03528a626c19/Sumstats/phecode-250.2-both_sexes.tsv.bgz"
!curl https://pan-ukb-us-east-1.s3.amazonaws.com/sumstats_flat_files/phecode-495-both_sexes.tsv.bgz | gsutil cp - "gs://fc-secure-9afe7562-2fad-4781-ab60-03528a626c19/Sumstats/phecode-495-both_sexes.tsv.bgz"
!curl https://pan-ukb-us-east-1.s3.amazonaws.com/sumstats_flat_files/phecode-174.1-females.tsv.bgz | gsutil cp - "gs://fc-secure-9afe7562-2fad-4781-ab60-03528a626c19/Sumstats/phecode-174.1-females.tsv.bgz"
!curl https://pan-ukb-us-east-1.s3.amazonaws.com/sumstats_flat_files/phecode-153-both_sexes.tsv.bgz | gsutil cp - "gs://fc-secure-9afe7562-2fad-4781-ab60-03528a626c19/Sumstats/phecode-153-both_sexes.tsv.bgz"

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
  0     0    0     0    0     0      0      0 --:--:-- --:--:-- --:--:--     0Copying from <STDIN>...
100 1953M  100 1953M    0     0  42.3M      0  0:00:46  0:00:46 --:--:-- 37.3M  / [0 files][264.0 KiB/    0.0 B]                                                / [0 files][100.0 MiB/    0.0 B]                                                / [0 files][200.1 MiB/    0.0 B]                                                / [0 files][300.1 MiB/    0.0 B]                                                / [0 files][400.1 MiB/    0.0 B]                                                / [0 files][500.2 MiB/    0.0 B]   30.8 MiB/s                                   / [0 files][600.2 MiB/    0.0 B]   35.0 MiB/s                                   / [0 files][700.2 MiB/    0.0 B]   41.4 MiB/s                                   / [0 files][800.2 Mi

100 2352M  100 2352M    0     0  42.1M      0  0:00:55  0:00:55 --:--:-- 56.6M  / [0 files][264.0 KiB/    0.0 B]                                                / [0 files][100.0 MiB/    0.0 B]                                                / [0 files][200.1 MiB/    0.0 B]                                                / [0 files][300.1 MiB/    0.0 B]                                                / [0 files][400.1 MiB/    0.0 B]                                                / [0 files][500.2 MiB/    0.0 B]   50.7 MiB/s                                   / [0 files][600.2 MiB/    0.0 B]   47.9 MiB/s                                   / [0 files][700.2 MiB/    0.0 B]   49.2 MiB/s                                   / [0 files][800.2 MiB/    0.0 B]   52.5 MiB/s                                   / [0 files][900.0 MiB/    0.0 B]   34.8 MiB/s                                   / [0 files][ 1000 MiB/    0.0 B]   34.1 MiB/s                                   / [0 files][  1.1 GiB/    0.0 B]   34.3 

## QC

1. Liftover, key the sumstats by locus
2. Restrict to biallelic variants 
3. Negative strand problems
4. Annotate to matrix table, and swap ref:alt if necessary

### Liftover, then key the sumstats by locus

In [4]:
rg37 = hl.get_reference('GRCh37')
rg38 = hl.get_reference('GRCh38')  
rg37.add_liftover('gs://hail-common/references/grch37_to_grch38.over.chain.gz', rg38)  

Initializing Hail with default parameters...
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
Running on Apache Spark version 3.1.3
SparkUI available at http://all-of-us-2936-m.c.terra-vpc-sc-fd39b54c.internal:33887
Welcome to
     __  __     <>__
    / /_/ /__  __/ /
   / __  / _ `/ / /
  /_/ /_/\_,_/_/_/   version 0.2.107-2387bb00ceee
LOGGING: writing to /home/jupyter/workspaces/prswithwgsvsarraydata/hail-20230323-2045-0.2.107-2387bb00ceee.log


In [ ]:
var_array = hl.read_table(f"{bucket}/ArrayData/Array_Vars_QCed.ht")
var_array = var_array.key_by("locus")

In [8]:
# write a function to streamline this
def sumstats_QC(filename_in, filename_temp, filename_out, var_array):
    sumstats = hl.import_table(filename_in, 
                               impute=True, 
                               types={"pos": "str"})
    # most of the fields are irrelevant for our analysis, drop it first to save computation 
    sumstats = sumstats.select(sumstats.chr, 
                               sumstats.pos,
                               sumstats.ref,
                               sumstats.alt,
                               sumstats.beta_meta, 
                               sumstats.se_meta, 
                               sumstats.neglog10_pval_meta)
    
    sumstats = sumstats.annotate(variant = sumstats['chr']+":"+sumstats['pos'] +":"+ sumstats['ref'] +":"+ sumstats['alt'])
    sumstats = sumstats.annotate(**hl.parse_variant(sumstats.variant, reference_genome='GRCh37'))
    sumstats = sumstats.annotate(**hl.liftover(sumstats.locus, 'GRCh38', include_strand=True))
    sumstats = sumstats.rename({'result' : 'locus_GRCh38',
                                'locus' : 'locus_GRCh37'})
    sumstats = sumstats.filter((hl.is_defined(sumstats.locus_GRCh38)) & 
                                      (~hl.is_nan(sumstats.beta_meta)) &
                                      (~hl.is_nan(sumstats.se_meta)) &
                                      (~hl.is_nan(sumstats.neglog10_pval_meta))) 
    sumstats = sumstats.key_by(sumstats.locus_GRCh38)
    
    
    
    ############## biallelic SNPs ############
    sumstats_loci_count = sumstats.group_by(*sumstats.key).aggregate(n_allele = hl.agg.count())
    sumstats = sumstats.annotate(**sumstats_loci_count[sumstats.locus_GRCh38])
    sumstats = sumstats.filter(sumstats.n_allele == 1)

    ## check if is_SNP
    sumstats = sumstats.filter(hl.is_snp(sumstats.alleles[0], sumstats.alleles[1]))
    
    ############## flip negative strand ############
    alleles_fix_neg_strand = hl.if_else(sumstats.is_negative_strand, 
           hl.reverse_complement(sumstats.alleles[0]) + ":" + hl.reverse_complement(sumstats.alleles[1]), 
           sumstats.alleles[0] + ":" + sumstats.alleles[1]).split(':')

    sumstats = sumstats.annotate(alleles_fix_neg_strand = alleles_fix_neg_strand)
    sumstats = sumstats.checkpoint(filename_temp) 
    
    ############## check ref:alt, flip if necessary ############
    var_array = var_array.annotate(**sumstats[var_array.locus])
    var_array = var_array.filter((~hl.is_nan(var_array.beta_meta)) &
                                  (~hl.is_nan(var_array.se_meta)) &
                                  (~hl.is_nan(var_array.neglog10_pval_meta)))
    
    var_array = var_array.annotate(beta_meta_fix_ref_alt = hl.case()
        .when(((var_array.alleles[0] == var_array.alleles_fix_neg_strand[0]) & (var_array.alleles[1] == var_array.alleles_fix_neg_strand[1])), var_array.beta_meta) 
        .when(((var_array.alleles[0] == var_array.alleles_fix_neg_strand[1]) & (var_array.alleles[1] == var_array.alleles_fix_neg_strand[0])), -var_array.beta_meta)
        .default(float('nan')))
    
    sumstats_QCed = var_array.select(
                        rsid = var_array.rsid,
                        alleles1_array = var_array.alleles[0],
                        alleles2_array = var_array.alleles[1],
                        alleles1_sumstats_original = var_array.ref,
                        alleles2_sumstats_original = var_array.alt,
                        is_negative_strand = var_array.is_negative_strand,
                        alleles1_sumstats_fixstrand = var_array.alleles_fix_neg_strand[0],
                        alleles2_sumstats_fixstrand = var_array.alleles_fix_neg_strand[1],
                        beta_meta = var_array.beta_meta,
                        beta_meta_fix_ref_alt = var_array.beta_meta_fix_ref_alt,
                        se_meta = var_array.se_meta,
                        neglog10_pval_meta = var_array.neglog10_pval_meta)
    
    sumstats_QCed.export(filename_out)

### Note:

Binary phenotype sumstats doesn't have `af_meta_hq`, `beta_meta_hq`,`se_meta_hq`,`neglog10_pval_meta_hq` fields;  

Therefore, used `beta_meta`,`se_meta`,`neglog10_pval_meta` fields for binary traits

Moreover, binary traits split `af` by case vs control; didn't include AF fields in QC pipeline for binary traits 

In [7]:
# 5 worker + 0 preeemptible

print("Height " + str(datetime.now()))
sumstats_QC(f"{bucket}/Sumstats/continuous-50-both_sexes-irnt.tsv.bgz",
            f"{bucket}/Sumstats/continuous-50-both_sexes-irnt_checkpoint.ht",
            f"{bucket}/Sumstats/Array_Height_QCed.tsv.bgz",
            var_array)

print("DBP " + str(datetime.now()))
sumstats_QC(f"{bucket}/Sumstats/continuous-DBP-both_sexes-auto_medadj_irnt.tsv.bgz",
            f"{bucket}/Sumstats/continuous-DBP-both_sexes-auto_medadj_irnt_checkpoint.ht",
            f"{bucket}/Sumstats/Array_DBP_QCed.tsv.bgz",
            var_array)

print("HDL " + str(datetime.now()))
sumstats_QC(f"{bucket}/Sumstats/biomarkers-30760-both_sexes-irnt.tsv.bgz",
            f"{bucket}/Sumstats/biomarkers-30760-both_sexes-irnt_checkpoint.ht",
            f"{bucket}/Sumstats/Array_HDL_QCed.tsv.bgz",
            var_array)

print("TC " + str(datetime.now()))
sumstats_QC(f"{bucket}/Sumstats/biomarkers-30690-both_sexes-irnt.tsv.bgz",
            f"{bucket}/Sumstats/biomarkers-30690-both_sexes-irnt_checkpoint.ht",
            f"{bucket}/Sumstats/Array_TC_QCed.tsv.bgz",
            var_array)

print("RBC " + str(datetime.now()))
sumstats_QC(f"{bucket}/Sumstats/continuous-30010-both_sexes-irnt.tsv.bgz",
            f"{bucket}/Sumstats/continuous-30010-both_sexes-irnt_checkpoint.ht",
            f"{bucket}/Sumstats/Array_RBC_QCed.tsv.bgz",
            var_array)

print("leukocyte " + str(datetime.now()))
sumstats_QC(f"{bucket}/Sumstats/continuous-30000-both_sexes-irnt.tsv.bgz",
            f"{bucket}/Sumstats/continuous-30000-both_sexes-irnt_checkpoint.ht",
            f"{bucket}/Sumstats/Array_leukocyte_QCed.tsv.bgz",
            var_array)

Height 2023-03-23 16:06:02.239457


2023-03-23 16:06:16.110 Hail: INFO: wrote table with 1949630 rows in 1 partition to /tmp/persist_tableJz7t06OTFL
2023-03-23 16:06:19.700 Hail: INFO: Reading table to impute column types 1) / 1]
2023-03-23 16:07:21.721 Hail: INFO: Loading <StructExpression of type struct{chr: str, pos: str, ref: str, alt: str, af_meta_hq: float64, beta_meta_hq: float64, se_meta_hq: float64, neglog10_pval_meta_hq: float64, neglog10_pval_heterogeneity_hq: float64, af_meta: float64, beta_meta: float64, se_meta: float64, neglog10_pval_meta: float64, neglog10_pval_heterogeneity: float64, af_AFR: float64, af_AMR: float64, af_CSA: float64, af_EAS: float64, af_EUR: float64, beta_AFR: float64, beta_AMR: float64, beta_CSA: float64, beta_EAS: float64, beta_EUR: float64, se_AFR: float64, se_AMR: float64, se_CSA: float64, se_EAS: float64, se_EUR: float64, neglog10_pval_AFR: float64, neglog10_pval_AMR: float64, neglog10_pval_CSA: float64, neglog10_pval_EAS: float64, neglog10_pval_EUR: float64, low_confidence_AFR: boo

DBP 2023-03-23 16:14:29.030924


2023-03-23 16:14:39.525 Hail: INFO: wrote table with 1725455 rows in 1 partition to /tmp/persist_tablekjTZXwdKxn
2023-03-23 16:14:40.153 Hail: INFO: Reading table to impute column types
2023-03-23 16:15:38.928 Hail: INFO: Loading <StructExpression of type struct{chr: str, pos: str, ref: str, alt: str, af_meta_hq: float64, beta_meta_hq: float64, se_meta_hq: float64, neglog10_pval_meta_hq: float64, neglog10_pval_heterogeneity_hq: float64, af_meta: float64, beta_meta: float64, se_meta: float64, neglog10_pval_meta: float64, neglog10_pval_heterogeneity: float64, af_AFR: float64, af_AMR: float64, af_CSA: float64, af_EAS: float64, af_EUR: float64, af_MID: float64, beta_AFR: float64, beta_AMR: float64, beta_CSA: float64, beta_EAS: float64, beta_EUR: float64, beta_MID: float64, se_AFR: float64, se_AMR: float64, se_CSA: float64, se_EAS: float64, se_EUR: float64, se_MID: float64, neglog10_pval_AFR: float64, neglog10_pval_AMR: float64, neglog10_pval_CSA: float64, neglog10_pval_EAS: float64, neglog

HDL 2023-03-23 16:22:25.773188


...

leukocyte 2023-03-23 16:46:26.685795


2023-03-23 16:46:39.413 Hail: INFO: wrote table with 1725037 rows in 1 partition to /tmp/persist_tablenQ0ixFrlH4
2023-03-23 16:46:40.898 Hail: INFO: Reading table to impute column types 1) / 1]
2023-03-23 16:47:33.858 Hail: INFO: Loading <StructExpression of type struct{chr: str, pos: str, ref: str, alt: str, af_meta_hq: float64, beta_meta_hq: float64, se_meta_hq: float64, neglog10_pval_meta_hq: float64, neglog10_pval_heterogeneity_hq: float64, af_meta: float64, beta_meta: float64, se_meta: float64, neglog10_pval_meta: float64, neglog10_pval_heterogeneity: float64, af_AFR: float64, af_AMR: float64, af_CSA: float64, af_EAS: float64, af_EUR: float64, af_MID: float64, beta_AFR: float64, beta_AMR: float64, beta_CSA: float64, beta_EAS: float64, beta_EUR: float64, beta_MID: float64, se_AFR: float64, se_AMR: float64, se_CSA: float64, se_EAS: float64, se_EUR: float64, se_MID: float64, neglog10_pval_AFR: float64, neglog10_pval_AMR: float64, neglog10_pval_CSA: float64, neglog10_pval_EAS: float64

T2D 2023-03-23 16:53:57.204369


2023-03-23 16:54:09.798 Hail: INFO: wrote table with 1626966 rows in 1 partition to /tmp/persist_table7OrCT88jwW
2023-03-23 16:54:11.088 Hail: INFO: Reading table to impute column types 1) / 1]
2023-03-23 16:55:13.536 Hail: INFO: Loading <StructExpression of type struct{chr: str, pos: str, ref: str, alt: str, af_cases_meta_hq: float64, af_controls_meta_hq: float64, beta_meta_hq: float64, se_meta_hq: float64, neglog10_pval_meta_hq: float64, neglog10_pval_heterogeneity_hq: float64, af_cases_meta: float64, af_controls_meta: float64, beta_meta: float64, se_meta: float64, neglog10_pval_meta: float64, neglog10_pval_heterogeneity: float64, af_cases_AFR: float64, af_cases_CSA: float64, af_cases_EAS: float64, af_cases_EUR: float64, af_cases_MID: float64, af_controls_AFR: float64, af_controls_CSA: float64, af_controls_EAS: float64, af_controls_EUR: float64, af_controls_MID: float64, beta_AFR: float64, beta_CSA: float64, beta_EAS: float64, beta_EUR: float64, beta_MID: float64, se_AFR: float64, se

AttributeError: Table instance has no field, method, or property 'af_meta_hq'
    Did you mean:
        Data fields: 'beta_meta_hq' [row], 'se_meta_hq' [row], 'af_cases_meta_hq' [row], 'af_controls_meta_hq' [row], 'beta_meta' [row]

In [13]:
print("T2D " + str(datetime.now()))
sumstats_QC(f"{bucket}/Sumstats/phecode-250.2-both_sexes.tsv.bgz",
            f"{bucket}/Sumstats/phecode-250.2-both_sexes_checkpoint.ht",
            f"{bucket}/Sumstats/Array_T2D_QCed.tsv.bgz",
            var_array)

print("Asthma " + str(datetime.now()))
sumstats_QC(f"{bucket}/Sumstats/phecode-495-both_sexes.tsv.bgz",
            f"{bucket}/Sumstats/phecode-495-both_sexes_checkpoint.ht",
            f"{bucket}/Sumstats/Array_Asthma_QCed.tsv.bgz",
            var_array)

print("Breast_Cancer " + str(datetime.now()))
sumstats_QC(f"{bucket}/Sumstats/phecode-174.1-females.tsv.bgz",
            f"{bucket}/Sumstats/phecode-174.1-females_checkpoing.ht",
            f"{bucket}/Sumstats/Array_Breast_Cancer_QCed.tsv.bgz",
            var_array)

print("Colorectal_Cancer " + str(datetime.now()))
sumstats_QC(f"{bucket}/Sumstats/phecode-153-both_sexes.tsv.bgz",
            f"{bucket}/Sumstats/phecode-153-both_sexes_checkpoint.ht",
            f"{bucket}/Sumstats/Array_Colorectal_Cancer_QCed.tsv.bgz",
            var_array)

T2D 2023-03-23 20:56:11.491241


2023-03-23 20:56:24.264 Hail: INFO: wrote table with 1626966 rows in 1 partition to /tmp/persist_tableGvlWIPGdE0
2023-03-23 20:56:24.900 Hail: INFO: Reading table to impute column types
2023-03-23 20:57:26.676 Hail: INFO: Loading <StructExpression of type struct{chr: str, pos: str, ref: str, alt: str, af_cases_meta_hq: float64, af_controls_meta_hq: float64, beta_meta_hq: float64, se_meta_hq: float64, neglog10_pval_meta_hq: float64, neglog10_pval_heterogeneity_hq: float64, af_cases_meta: float64, af_controls_meta: float64, beta_meta: float64, se_meta: float64, neglog10_pval_meta: float64, neglog10_pval_heterogeneity: float64, af_cases_AFR: float64, af_cases_CSA: float64, af_cases_EAS: float64, af_cases_EUR: float64, af_cases_MID: float64, af_controls_AFR: float64, af_controls_CSA: float64, af_controls_EAS: float64, af_controls_EUR: float64, af_controls_MID: float64, beta_AFR: float64, beta_CSA: float64, beta_EAS: float64, beta_EUR: float64, beta_MID: float64, se_AFR: float64, se_CSA: fl

Asthma 2023-03-23 21:04:12.377135


2023-03-23 21:04:23.914 Hail: INFO: wrote table with 1627642 rows in 1 partition to /tmp/persist_tables7M5yslARO
2023-03-23 21:04:25.326 Hail: INFO: Reading table to impute column types 1) / 1]
2023-03-23 21:05:24.166 Hail: INFO: Loading <StructExpression of type struct{chr: str, pos: str, ref: str, alt: str, af_cases_meta: float64, af_controls_meta: float64, beta_meta: float64, se_meta: float64, neglog10_pval_meta: float64, neglog10_pval_heterogeneity: float64, af_cases_AFR: float64, af_cases_AMR: float64, af_cases_CSA: float64, af_cases_EAS: float64, af_cases_EUR: float64, af_cases_MID: float64, af_controls_AFR: float64, af_controls_AMR: float64, af_controls_CSA: float64, af_controls_EAS: float64, af_controls_EUR: float64, af_controls_MID: float64, beta_AFR: float64, beta_AMR: float64, beta_CSA: float64, beta_EAS: float64, beta_EUR: float64, beta_MID: float64, se_AFR: float64, se_AMR: float64, se_CSA: float64, se_EAS: float64, se_EUR: float64, se_MID: float64, neglog10_pval_AFR: floa

Breast_Cancer 2023-03-23 21:12:05.188041


2023-03-23 21:12:18.111 Hail: INFO: wrote table with 2081246 rows in 1 partition to /tmp/persist_tablec7QXZ99EfG
2023-03-23 21:12:19.432 Hail: INFO: Reading table to impute column types 1) / 1]
2023-03-23 21:13:14.167 Hail: INFO: Loading <StructExpression of type struct{chr: str, pos: str, ref: str, alt: str, af_cases_meta: float64, af_controls_meta: float64, beta_meta: float64, se_meta: float64, neglog10_pval_meta: float64, neglog10_pval_heterogeneity: float64, af_cases_AFR: float64, af_cases_CSA: float64, af_cases_EAS: float64, af_cases_EUR: float64, af_controls_AFR: float64, af_controls_CSA: float64, af_controls_EAS: float64, af_controls_EUR: float64, beta_AFR: float64, beta_CSA: float64, beta_EAS: float64, beta_EUR: float64, se_AFR: float64, se_CSA: float64, se_EAS: float64, se_EUR: float64, neglog10_pval_AFR: float64, neglog10_pval_CSA: float64, neglog10_pval_EAS: float64, neglog10_pval_EUR: float64, low_confidence_AFR: bool, low_confidence_CSA: bool, low_confidence_EAS: bool, low

Colorectal_Cancer 2023-03-23 21:20:11.065135


2023-03-23 21:20:23.893 Hail: INFO: wrote table with 2415301 rows in 1 partition to /tmp/persist_table40xy2auDWl
2023-03-23 21:20:25.171 Hail: INFO: Reading table to impute column types 1) / 1]
2023-03-23 21:21:22.050 Hail: INFO: Finished type imputation=>    (11 + 1) / 12]
  Loading field 'chr' as type str (imputed)
  Loading field 'pos' as type str (user-supplied type)
  Loading field 'ref' as type str (imputed)
  Loading field 'alt' as type str (imputed)
  Loading field 'af_cases_meta' as type float64 (imputed)
  Loading field 'af_controls_meta' as type float64 (imputed)
  Loading field 'beta_meta' as type float64 (imputed)
  Loading field 'se_meta' as type float64 (imputed)
  Loading field 'neglog10_pval_meta' as type float64 (imputed)
  Loading field 'neglog10_pval_heterogeneity' as type float64 (imputed)
  Loading field 'af_cases_AFR' as type float64 (imputed)
  Loading field 'af_cases_CSA' as type float64 (imputed)
  Loading field 'af_cases_EUR' as type float64 (imputed)
  Loadi